In [1]:
import pandas as pd
import pickle

menwomen = ['Men','Women']
mwcsv = ['olympicsoccer24men.csv','olympicsoccer24women.csv']

mw = 1

In [2]:
# use pdf from olympics to get names and teams

import pandas as pd
import pickle



if True:
    # Step 1: Read the CSV file without headers
    df = pd.read_csv(mwcsv[mw], header=None)
    
    # Step 2: Assign column names
    # Adjust based on the actual number of columns in the CSV
    df.columns = ['Team', 'Name']
    
    # Step 3: Drop rows with any missing or empty values in 'Team' or 'Name'
    df = df.dropna(subset=['Team', 'Name'])  # Remove rows with NaN values in 'Team' or 'Name'
    df = df[(df['Team'].str.strip() != '') & (df['Name'].str.strip() != '')]  # Remove rows with empty strings
    
    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))
    
# Display the cleaned DataFrame
print(df)


          Team                 Name
0         Team                 Name
1    Australia     ARNOLD Mackenzie
2    Australia      HEYMAN Michelle
3    Australia       TORPEY Kaitlyn
4    Australia   POLKINGHORNE Clare
..         ...                  ...
271     Zambia        MUSOLE Ngambo
272     Zambia           PHIRI Vast
273     Zambia      NACHULA Racheal
274     Zambia         WILOMBE Mary
275     Zambia        SAKALA Eunice

[265 rows x 2 columns]


In [3]:
# fix the women names
if mw == 1:
    df['Name'] = df['Name'].apply(lambda x: ' '.join(reversed(x.title().split())))
    print(df.head())


        Team                Name
0       Team                Name
1  Australia    Mackenzie Arnold
2  Australia     Michelle Heyman
3  Australia      Kaitlyn Torpey
4  Australia  Clare Polkinghorne


In [4]:
# get wiki url for players

import pandas as pd
import requests
import time

if True:

    # Step 4: Function to get Wikipedia page URL
    def get_wikipedia_page_url(name):
        search_url = "https://en.wikipedia.org/w/api.php"
        params = {
            'action': 'query',
            'format': 'json',
            'list': 'search',
            'srsearch': name,
            'utf8': 1,
            'srlimit': 1
        }
        response = requests.get(search_url, params=params)
        data = response.json()
        
        if 'query' in data and 'search' in data['query']:
            search_results = data['query']['search']
            if search_results:
                page_title = search_results[0]['title']
                page_url = f"https://en.wikipedia.org/wiki/{page_title.replace(' ', '_')}"
                return page_url
        return None
    
    # Step 5: Add Wikipedia URLs to DataFrame
    df['Name_Link'] = df['Name'].apply(lambda name: get_wikipedia_page_url(name) or None)
    
    # Add delay to avoid rate limiting
    time.sleep(1)

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))

# Display the DataFrame with Wikipedia URLs
print(df)


          Team                Name  \
0         Team                Name   
1    Australia    Mackenzie Arnold   
2    Australia     Michelle Heyman   
3    Australia      Kaitlyn Torpey   
4    Australia  Clare Polkinghorne   
..         ...                 ...   
271     Zambia       Ngambo Musole   
272     Zambia          Vast Phiri   
273     Zambia     Racheal Nachula   
274     Zambia        Mary Wilombe   
275     Zambia       Eunice Sakala   

                                            Name_Link  
0                  https://en.wikipedia.org/wiki/Name  
1      https://en.wikipedia.org/wiki/Mackenzie_Arnold  
2       https://en.wikipedia.org/wiki/Michelle_Heyman  
3        https://en.wikipedia.org/wiki/Kaitlyn_Torpey  
4    https://en.wikipedia.org/wiki/Clare_Polkinghorne  
..                                                ...  
271       https://en.wikipedia.org/wiki/Ngambo_Musole  
272          https://en.wikipedia.org/wiki/Vast_Phiri  
273     https://en.wikipedia.org/wiki/R

In [5]:
print(df['Name_Link'].isna().sum())

1


In [6]:
# check if webpage is good if not make url=None

import pandas as pd

# Function to validate URLs
def validate_url(row):
    name = row['Name']
    url = row['Name_Link']
    
    if pd.isna(url):
        return None
    
    # Extract the part after the last '/'
    url_name_part = url.rsplit('/', 1)[-1]
    
    # Compare the first 3 letters
    if url_name_part[:3].lower() == name[:3].lower():
        return url
    else:
        return None

# Apply the validation function to each row
df['Name_Link'] = df.apply(validate_url, axis=1)

# Display the updated DataFrame
print(df)


          Team                Name  \
0         Team                Name   
1    Australia    Mackenzie Arnold   
2    Australia     Michelle Heyman   
3    Australia      Kaitlyn Torpey   
4    Australia  Clare Polkinghorne   
..         ...                 ...   
271     Zambia       Ngambo Musole   
272     Zambia          Vast Phiri   
273     Zambia     Racheal Nachula   
274     Zambia        Mary Wilombe   
275     Zambia       Eunice Sakala   

                                            Name_Link  
0                  https://en.wikipedia.org/wiki/Name  
1      https://en.wikipedia.org/wiki/Mackenzie_Arnold  
2       https://en.wikipedia.org/wiki/Michelle_Heyman  
3        https://en.wikipedia.org/wiki/Kaitlyn_Torpey  
4    https://en.wikipedia.org/wiki/Clare_Polkinghorne  
..                                                ...  
271       https://en.wikipedia.org/wiki/Ngambo_Musole  
272          https://en.wikipedia.org/wiki/Vast_Phiri  
273     https://en.wikipedia.org/wiki/R

In [7]:
df['Name_Link'].isna().sum()

26

In [8]:
# make sure web pages are ok

import pandas as pd

# Function to validate URLs
def validate_url(row):
    name = row['Name']
    url = row['Name_Link']
    
    if pd.isna(url):
        return None
    
    # Extract the part after the last '/'
    url_name_part = url.rsplit('/', 1)[-1]
    
    # Compare the first 3 letters
    if url_name_part[:3].lower() == name[:3].lower():
        return url
    else:
        return None

# Apply the validation function to each row
df['Name_Link'] = df.apply(validate_url, axis=1)

# Display the updated DataFrame
print(df)


          Team                Name  \
0         Team                Name   
1    Australia    Mackenzie Arnold   
2    Australia     Michelle Heyman   
3    Australia      Kaitlyn Torpey   
4    Australia  Clare Polkinghorne   
..         ...                 ...   
271     Zambia       Ngambo Musole   
272     Zambia          Vast Phiri   
273     Zambia     Racheal Nachula   
274     Zambia        Mary Wilombe   
275     Zambia       Eunice Sakala   

                                            Name_Link  
0                  https://en.wikipedia.org/wiki/Name  
1      https://en.wikipedia.org/wiki/Mackenzie_Arnold  
2       https://en.wikipedia.org/wiki/Michelle_Heyman  
3        https://en.wikipedia.org/wiki/Kaitlyn_Torpey  
4    https://en.wikipedia.org/wiki/Clare_Polkinghorne  
..                                                ...  
271       https://en.wikipedia.org/wiki/Ngambo_Musole  
272          https://en.wikipedia.org/wiki/Vast_Phiri  
273     https://en.wikipedia.org/wiki/R

In [9]:
df.to_csv('cleaned_olympicsoccer.csv', index=False)

In [10]:
# read in capitals

df_capitals = pd.read_csv('capitals.csv', encoding='latin1')
df_capitals.head()

,Country,Capital City,Latitude,Longitude,Population,Capital Type
0,Afghanistan,Kabul,34.5289,69.1725,4011770,Capital
1,Albania,Tiranë (Tirana),41.3275,19.8189,475577,Capital
2,Algeria,El Djazaïr (Algiers),36.7525,3.0420,2693542,Capital
3,American Samoa,Pago Pago,-14.2781,-170.7025,48526,Capital
4,Andorra,Andorra la Vella,42.5078,1.5211,22614,Capital


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

if True:

    # Function to extract details from Wikipedia infobox
    def extract_infobox_details(url):
        if url is None:
            return None, None, None
        
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            infobox = soup.find('table', {'class': 'infobox'})
            
            if infobox is None:
                return None, None, None
            
            # Extracting photo link
            img_tag = infobox.find('img')
            photo_link = 'https:' + img_tag['src'] if img_tag else None
            
            # Extracting birthplace and birthplace link
            birthplace_data = infobox.find('th', string='Place of birth')
            if birthplace_data is None:
                return None, None, photo_link  # No birthplace data, return photo link
            
            birthplace_data = birthplace_data.find_next_sibling('td')
            birthplace = birthplace_data.get_text(" ", strip=True)
            birthplace_links = birthplace_data.find_all('a')
            birthplace_link = ['https://en.wikipedia.org' + a['href'] for a in birthplace_links]
            
            return birthplace, birthplace_link, photo_link
        except Exception as e:
            return None, None, f'Error: {e}'
    
    # Apply the function to each URL in the DataFrame
    def process_row(row):
        birthplace, birthplace_link, photo_link = extract_infobox_details(row['Name_Link'])
        # Handle None for birthplace and links
        if birthplace is None:
            birthplace = None
        if birthplace_link is None:
            birthplace_link = None
        return pd.Series([birthplace, birthplace_link, photo_link], index=['birthplace', 'birthplace_link', 'photo_link'])
    
    # Apply function and add new columns to the DataFrame
    df[['birthplace', 'birthplace_link', 'photo_Link']] = df.apply(process_row, axis=1)

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))

# Set pandas options to display full content
pd.set_option('display.max_colwidth', None)  # Ensure full content is displayed

# Print the DataFrame to see all details
print(df)


          Team                Name  \
0         Team                Name   
1    Australia    Mackenzie Arnold   
2    Australia     Michelle Heyman   
3    Australia      Kaitlyn Torpey   
4    Australia  Clare Polkinghorne   
..         ...                 ...   
271     Zambia       Ngambo Musole   
272     Zambia          Vast Phiri   
273     Zambia     Racheal Nachula   
274     Zambia        Mary Wilombe   
275     Zambia       Eunice Sakala   

                                            Name_Link  \
0                  https://en.wikipedia.org/wiki/Name   
1      https://en.wikipedia.org/wiki/Mackenzie_Arnold   
2       https://en.wikipedia.org/wiki/Michelle_Heyman   
3        https://en.wikipedia.org/wiki/Kaitlyn_Torpey   
4    https://en.wikipedia.org/wiki/Clare_Polkinghorne   
..                                                ...   
271       https://en.wikipedia.org/wiki/Ngambo_Musole   
272          https://en.wikipedia.org/wiki/Vast_Phiri   
273     https://en.wikipedia.o

In [12]:
df.columns

Index(['Team', 'Name', 'Name_Link', 'birthplace', 'birthplace_link',
       'photo_Link'],
      dtype='object')

In [13]:
# get summary

import pandas as pd
import requests
import time

if True:

  # Function to fetch Wikipedia page summary
  def fetch_wikipedia_summary(link):
      time.sleep(0.2)  # Add a delay of 0.2 seconds between requests
      try:
          # Extract page title from the link
          title = link.split('/')[-1]
          url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
          response = requests.get(url)
          data = response.json()
          if 'extract' in data:
              return data['extract']
      except Exception as e:
          None #print(f"Error fetching summary for {link}: {e}")
      return None

  # Apply the function to each row in the DataFrame
  df['summary'] = df['Name_Link'].apply(fetch_wikipedia_summary)


  # Pickle the DataFrame
  pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))

else:

  # Load the pickled DataFrame
  df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))

print(df)




          Team                Name  \
0         Team                Name   
1    Australia    Mackenzie Arnold   
2    Australia     Michelle Heyman   
3    Australia      Kaitlyn Torpey   
4    Australia  Clare Polkinghorne   
..         ...                 ...   
271     Zambia       Ngambo Musole   
272     Zambia          Vast Phiri   
273     Zambia     Racheal Nachula   
274     Zambia        Mary Wilombe   
275     Zambia       Eunice Sakala   

                                            Name_Link  \
0                  https://en.wikipedia.org/wiki/Name   
1      https://en.wikipedia.org/wiki/Mackenzie_Arnold   
2       https://en.wikipedia.org/wiki/Michelle_Heyman   
3        https://en.wikipedia.org/wiki/Kaitlyn_Torpey   
4    https://en.wikipedia.org/wiki/Clare_Polkinghorne   
..                                                ...   
271       https://en.wikipedia.org/wiki/Ngambo_Musole   
272          https://en.wikipedia.org/wiki/Vast_Phiri   
273     https://en.wikipedia.o

In [14]:
# get coordinates

import pandas as pd
import requests
import pickle

if True: 
    # Function to fetch coordinates from Wikipedia API with error handling
    def fetch_coordinates_from_wikipedia(link):
        try:
            # Extract page title from the link
            title = link.split('/')[-1]
            url = f"https://en.wikipedia.org/w/api.php?action=query&prop=coordinates&format=json&titles={title}"
            response = requests.get(url)
            data = response.json()
            pages = data.get('query', {}).get('pages', {})
            if pages:
                page = next(iter(pages.values()))  # Get the first page (should be only one)
                coords_list = page.get('coordinates', [])
                if coords_list:
                    # Get the first item in the coordinates list
                    coords = coords_list[0]
                    return (coords.get('lat'), coords.get('lon'))
            # If there are no pages or coordinates, return None
            return None
        except Exception as e:
            # Uncomment for debugging
            # print(f"Error fetching coordinates for {link}: {e}")
            return None
    
    # Assuming df['birthplace_link'] contains lists of URLs
    def get_first_coords_from_list(row):
        if isinstance(row, list) and row:
            first_url = row[0]
            return fetch_coordinates_from_wikipedia(first_url)
        return None
    
    # Example DataFrame creation
    # df = pd.DataFrame({'birthplace_link': [['https://en.wikipedia.org/wiki/Some_City'], ['https://en.wikipedia.org/wiki/Another_City']]})
    
    # Apply the function to get coordinates for the first URL in the list
    df['birth_coords'] = df['birthplace_link'].apply(get_first_coords_from_list)

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/soccerdf.pkl', 'wb'))

else:

    # Load the pickled DataFrame
    df = pickle.load(open('pkls/soccerdf.pkl', 'rb'))



print(df)


          Team                Name  \
0         Team                Name   
1    Australia    Mackenzie Arnold   
2    Australia     Michelle Heyman   
3    Australia      Kaitlyn Torpey   
4    Australia  Clare Polkinghorne   
..         ...                 ...   
271     Zambia       Ngambo Musole   
272     Zambia          Vast Phiri   
273     Zambia     Racheal Nachula   
274     Zambia        Mary Wilombe   
275     Zambia       Eunice Sakala   

                                            Name_Link  \
0                  https://en.wikipedia.org/wiki/Name   
1      https://en.wikipedia.org/wiki/Mackenzie_Arnold   
2       https://en.wikipedia.org/wiki/Michelle_Heyman   
3        https://en.wikipedia.org/wiki/Kaitlyn_Torpey   
4    https://en.wikipedia.org/wiki/Clare_Polkinghorne   
..                                                ...   
271       https://en.wikipedia.org/wiki/Ngambo_Musole   
272          https://en.wikipedia.org/wiki/Vast_Phiri   
273     https://en.wikipedia.o

In [15]:
print(df['birth_coords'].isna().sum())
print(df.shape)

89
(265, 8)


In [16]:
# make capital if home not known

# Remove duplicate countries in df_capitals
df_capitals_unique = df_capitals.drop_duplicates(subset='Country')

# Create a mapping from Country to Coordinates
team_to_coords = df_capitals_unique.set_index('Country')[['Latitude', 'Longitude']].to_dict(orient='index')

# Function to fill missing coordinates
def fill_missing_coords(row):
    if row['birth_coords'] is None:
        team = row['Team']
        coords = team_to_coords.get(team)
        if coords:
            return (coords['Latitude'], coords['Longitude'])
    return row['birth_coords']

# Apply the function to fill missing coordinates
df['birth_coords'] = df.apply(fill_missing_coords, axis=1)

print(df)

          Team                Name  \
0         Team                Name   
1    Australia    Mackenzie Arnold   
2    Australia     Michelle Heyman   
3    Australia      Kaitlyn Torpey   
4    Australia  Clare Polkinghorne   
..         ...                 ...   
271     Zambia       Ngambo Musole   
272     Zambia          Vast Phiri   
273     Zambia     Racheal Nachula   
274     Zambia        Mary Wilombe   
275     Zambia       Eunice Sakala   

                                            Name_Link  \
0                  https://en.wikipedia.org/wiki/Name   
1      https://en.wikipedia.org/wiki/Mackenzie_Arnold   
2       https://en.wikipedia.org/wiki/Michelle_Heyman   
3        https://en.wikipedia.org/wiki/Kaitlyn_Torpey   
4    https://en.wikipedia.org/wiki/Clare_Polkinghorne   
..                                                ...   
271       https://en.wikipedia.org/wiki/Ngambo_Musole   
272          https://en.wikipedia.org/wiki/Vast_Phiri   
273     https://en.wikipedia.o

In [17]:
# save with correct gender

# Pickle the DataFrame
pickle.dump(df, open('pkls/soccer'+menwomen[mw]+'df.pkl', 'wb'))

In [68]:
df_all={}
for mw in range(2):
    df_all[menwomen[mw]] = pickle.load(open('pkls/soccer'+menwomen[mw]+'df.pkl', 'rb'))


In [44]:
print(df_all['Men']['Team'].unique())
print(df_all['Women']['Team'].unique())


['Argentina' 'Dominican Republic' 'Egypt' 'France' 'Guinea' 'Iraq'
 'Israel' 'Japan' 'Mali' 'Morocco' 'New Zealand' 'Paraguay' 'Spain'
 'Ukraine' 'USA' 'Uzbekistan']
['Team' 'Australia' 'Brazil' 'Canada' 'Colombia' 'France' 'Germany'
 'Japan' 'New Zealand' 'Nigeria' 'Spain' 'USA' 'Zambia']


In [46]:
# Main colors for each country/team with formal color codes
main_colors = {
    'Argentina': '#00A3E0',  # Light Blue
    'Dominican Republic': '#0033A0',  # Blue
    'Egypt': '#CF1124',  # Red
    'France': '#0055A4',  # Blue
    'Guinea': '#F1C300',  # Yellow
    'Iraq': '#007A33',  # Green
    'Israel': '#0033A0',  # Blue
    'Japan': '#BC002D',  # Red
    'Mali': '#F1C300',  # Yellow
    'Morocco': '#C8102E',  # Red
    'New Zealand': '#000000',  # Black
    'Paraguay': '#0033A0',  # Blue
    'Spain': '#AA0000',  # Red
    'Ukraine': '#005BBB',  # Blue
    'USA': '#B22234',  # Red
    'Uzbekistan': '#009B77',  # Turquoise
    'Australia': '#006400',  # Dark Green
    'Brazil': '#00A859',  # Green
    'Canada': '#FF0000',  # Red
    'Colombia': '#FCD116',  # Yellow
    'Germany': '#000000',  # Black
    'Nigeria': '#008753',  # Green
    'Zambia': '#006A4E'  # Green
}

In [48]:
# Common abbreviations for each country/team
abbreviations = {
    'Argentina': 'ARG',
    'Dominican Republic': 'DOM',
    'Egypt': 'EGY',
    'France': 'FRA',
    'Guinea': 'GUI',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Japan': 'JPN',
    'Mali': 'MLI',
    'Morocco': 'MAR',
    'New Zealand': 'NZL',
    'Paraguay': 'PAR',
    'Spain': 'ESP',
    'Ukraine': 'UKR',
    'USA': 'USA',
    'Uzbekistan': 'UZB',
    'Australia': 'AUS',
    'Brazil': 'BRA',
    'Canada': 'CAN',
    'Colombia': 'COL',
    'Germany': 'GER',
    'Nigeria': 'NGA',
    'Zambia': 'ZAM'
}

In [50]:
df_all['Men'].columns


Index(['Team', 'Name', 'Name_Link', 'birthplace', 'birthplace_link',
       'photo_Link', 'summary', 'birth_coords'],
      dtype='object')

In [56]:
# Add the Google Tag Manager and AdSense scripts to the map
analytics = """
<!-- Google tag (gtag.js) -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-2RB0G9NLE8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-2RB0G9NLE8');
</script>

<script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-4771147286272001"
     crossorigin="anonymous"></script>
"""

In [70]:
# add null image
for mw in range(2):
    df_all[menwomen[mw]]['photo_Link'].fillna('https://upload.wikimedia.org/wikipedia/commons/1/1d/Football_Pallo_valmiina-cropped.jpg', inplace=True)


C:\Users\efrie\AppData\Local\Temp\ipykernel_21708\2874016525.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all[menwomen[mw]]['photo_Link'].fillna('https://upload.wikimedia.org/wikipedia/commons/1/1d/Football_Pallo_valmiina-cropped.jpg', inplace=True)


In [72]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, LocateControl
from branca.element import Element

# Example colors and abbreviations for soccer teams (for demonstration)
soccer_colors = {
    'Argentina': '#00A3E0',
    'Dominican Republic': '#0033A0',
    'Egypt': '#CF1124',
    'France': '#0055A4',
    'Guinea': '#F1C300',
    'Iraq': '#007A33',
    'Israel': '#0033A0',
    'Japan': '#BC002D',
    'Mali': '#F1C300',
    'Morocco': '#C8102E',
    'New Zealand': '#000000',
    'Paraguay': '#0033A0',
    'Spain': '#AA0000',
    'Ukraine': '#005BBB',
    'USA': '#B22234',
    'Uzbekistan': '#009B77',
    'Australia': '#006400',
    'Brazil': '#00A859',
    'Canada': '#FF0000',
    'Colombia': '#FCD116',
    'Germany': '#000000',
    'Nigeria': '#008753',
    'Zambia': '#006A4E'
}

soccer_abbreviations = {
    'Argentina': 'ARG',
    'Dominican Republic': 'DOM',
    'Egypt': 'EGY',
    'France': 'FRA',
    'Guinea': 'GUI',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Japan': 'JPN',
    'Mali': 'MLI',
    'Morocco': 'MAR',
    'New Zealand': 'NZL',
    'Paraguay': 'PAR',
    'Spain': 'ESP',
    'Ukraine': 'UKR',
    'USA': 'USA',
    'Uzbekistan': 'UZB',
    'Australia': 'AUS',
    'Brazil': 'BRA',
    'Canada': 'CAN',
    'Colombia': 'COL',
    'Germany': 'GER',
    'Nigeria': 'NGA',
    'Zambia': 'ZAM'
}

# Initialize map centered on the world
m = folium.Map(location=[20, 0], zoom_start=2)

# Add LocateControl to the map
LocateControl().add_to(m)

# Create MarkerCluster for soccer players
marker_cluster = MarkerCluster(name="Soccer Players")

# Add markers for each player in Men and Women dataframes
for gender in ['Men', 'Women']:
    df = df_all[gender]

    for index, row in df.iterrows():
        coords = row['birth_coords']
        team_name = row['Team']
        player_name = row['Name']
        birthplace = row['birthplace'] or 'Birthplace not known'
        
        # Determine the color and abbreviation for the team
        team_color = soccer_colors.get(team_name, '#FFFFFF')  # Default to white if not found
        team_abbreviation = soccer_abbreviations.get(team_name, 'N/A')
        
        # Create a div icon with colored circle and team abbreviation
        icon = folium.DivIcon(html=f'<div style="background-color: {team_color}; border-radius: 50%; '
                                   f'width: 30px; height: 30px; display: flex; justify-content: '
                                   f'center; align-items: center; color: white; font-weight: bold;">'
                                   f'{team_abbreviation}</div>')

        # Format popup content with wider width
        popup_content = f"""
        <div style="width: 300px;">
            <img src='{row['photo_Link']}' width='100px'><br>
            <b><big>{player_name.upper()}</big></b><br>
            {team_name} ({gender})<br>
            Birthplace: {birthplace}<br><br>
            {row['summary']}<br><br>
            <a href='{row['Name_Link']}' target='_blank'>Wikipedia Link</a>
        </div>
        """

        # Check if coords is None or not
        if coords is not None:
            # Create marker and add to MarkerCluster
            marker = folium.Marker([coords[0], coords[1]], popup=popup_content, icon=icon)
            marker.add_to(marker_cluster)

# Add MarkerCluster to the map
marker_cluster.add_to(m)

# Add the Google Analytics tracking code to the map
element = Element(analytics)
m.get_root().html.add_child(element)

# Save the map as HTML file
path = 'maps'
map_filename = 'map_soccer_players.html'
m.save(f"{path}/{map_filename}")

# Display the map (if in a Jupyter notebook or similar environment)
m
